In [ ]:
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import RandomizedSearchCV,train_test_split

train_df = pd.read_csv('../input/train_set.csv', sep='\t')
param_distribution = {
    "Ngram_range":[(1,2),(1,3),(2,3)],
    "Max_features": [1500,2000,2500,3000,3500],
}
train_data=train_df['text']
train_target=train_df['label']
train_num=160000
x_train,x_valid, y_train, y_valid =train_test_split(train_data,train_target,test_size=0.2, random_state=0)
def build_model(Ngram_range=(1,3),Max_features=3000):
    tfidf = TfidfVectorizer(ngram_range=Ngram_range,max_features=Max_features)
    train_test = tfidf.fit_transform(train_data)
    clf = RidgeClassifier()
    clf.fit(train_test[:train_num], train_df['label'].values[:train_num])
    return clf
sklearn_model=build_model()
random_search_cv = RandomizedSearchCV(sklearn_model,
                                      param_distribution,
                                      n_iter = 10,
                                      cv = 3,
                                      n_jobs = 1)                           
random_search_cv.fit(x_train, y_train, epochs = 100,
                        validation_data = (x_valid, y_valid),
                        )

In [ ]:
model = random_search_cv.best_estimator_.model
model.evaluate(x_test_scaled, y_test)

In [ ]:
val_pred = model.predict(train_test[train_num:])
print(f1_score(train_df['label'].values[train_num:], val_pred, average='macro'))